# Case Study
Denna notebook utgör den andra delen av funktionsanalysen som utförs inom ramarna av ett exjobb som genomförs under vårterminen 2025 av Elsa Hedström. Syftet med denna del är att teknik-bedömarna ska bilda sig en uppfattning om hur det är att lära sig att använda psycopg och PostGIS i vegetationsanalysen. Därefter kommer vegetationsanalysen att utvärderas för PostGIS (psycopg) respektive filgeodatabas (arcpy). 

Hela fallstudien uppskattas ta cirka 215 minuter.

## Implementation + Inläsning (PostGIS, psycopg)
*Tidsuppskattning: 100 minuter*

Vi börjar utvärderingen av lösningen som bygger på PostGIS och psycopg, vilket kommer att ske genom inläsning och implementation. 

Under detta moment kan ni läsa in er på PostGIS, PostgreSQL, psycopg och ogr2ogr parallellt med att ni testar paketet. Detta moment görs enskilt för att ni ska ta det i er egen takt och på det sätt som ni föredrar att lära er något nytt. Nedan listas dokumentation som kan vara användbar, men det begränsar er inte till att googla fritt efter ytterligare dokumentation.

- [PostgreSQL](https://www.postgresql.org/docs/current/)
- [PostGIS](https://postgis.net/docs/manual-3.5/en/)
- [psycopg](https://www.psycopg.org/docs/)
- [ogr2ogr](https://gdal.org/en/stable/programs/ogr2ogr.html)

För att bilda en uppfattning om hur lätt det är att lära sig PostGIS och implementera det med hjälp av psycopg ska ni utföra några delövningar som är moment i vegetationsanalysen. Ta hjälp om ni kör fast på någon övning, det viktiga här är att ni ska få en känsla för hur det är att lära sig de nya teknikerna med hjälp av befintlig dokumentation. 

### Uppgift 1: Sätt upp PostGIS databas
1. Använd psycopg för att ansluta till default databasen 'postgres'. Använd user='postgres', dbname='postgres' och lösenordet som du satte när du installerade PostgreSQL.
2. Skapa en ny databas som du döper till 'test_postgis_db'.
3. Anslut till den nya databasen.
4. Aktivera PostGIS-tillägg på den nya databasen.

In [1]:
import psycopg
from psycopg import sql
# Uppgift 1.1

# Uppgift 1.2

# Uppgift 1.3

# Uppgift 1.4

### Uppgift 2: Konvertera wires gdb till PostGIS
Till att börja med måste vi konvertera linorna från gdb till PostGIS. För detta krävs lite olika sökvägar från settings.json.
1. Kopiera in rätt sökväg för settings_file
2. Uppdatera sökvägarna för local_folder, powerline_list och powerlines_folder i settings.json.

In [11]:
# Uppgift 2.1
settings_file = r"C:/Users/SE1K4H/Desktop/case-study-material/settings.json"

2. Verifiera att *ogr2ogr_path, proj_lib_path, gdal_data_path* i settings.json stämmer överens med de faktiska sökvägarna.
3. Ladda in sökvägarna genom att köra nästa block.

In [ ]:
# Uppgift 2.3: Inga uppdateringar av koden krävs.
import json

with open(settings_file, 'r', encoding='utf-8') as file:
    settings = json.load(file)

run_ID = settings["run_ID"]
powerline_list = settings["powerline_list"]
local_dir = settings["local_folder"]
powerlines_folder = settings["powerlines_folder"]
ogr2ogr_path = settings["ogr2ogr_path"]   
proj_lib_path = settings["proj_lib_path"]
gdal_data_path = settings["gdal_data_path"]

print("Sökvägarna har lästs in")

4. Uppdatera variablerna nedan med rätt uppgifter för att ansluta till den databas som vi tidigare skapade. 

In [ ]:
# Uppgift 2.4
HOST = 'localhost'
DB_NAME = 'test_postgis_db'
USER = 'postgres'
PORT = '5432'
PASSWORD = ''

PG_CONNECTION = f"PG:host={HOST} dbname={DB_NAME} user={USER} password={PASSWORD} port={PORT}"

print(f"Strängen för att ansluta till databasen har lästs in: {PG_CONNECTION}")

5. Nu sker själva konverteringen av linorna med hjälp av verktyget ogr2ogr. Läs igenom koden och dokumentation för ogr2ogr för att bilda dig en uppfattning om vad som sker. Kör cellen. OBS: Det är viktigt att du har startat psql-servern i en cmd terminal i rätt sökväg.

    ```
    cd C:\Program Files\PostgreSQL\17\bin
    pg_ctl -D "C:\Program Files\PostgreSQL\17\data" start
    ```

In [ ]:
# Uppgift 2.5: Inga uppdateringar av koden krävs.
import os
import subprocess
import pandas as pd

def convert_gdb_to_postGIS(row):
    LG = row["LG"]
    line = row["line"]  
    table_name = f"{LG}_{line}_fas"
    
    ogr_command = [
        ogr2ogr_path,
        "-f", "PostgreSQL",
        PG_CONNECTION,
        wires_gdb,
        "-a_srs", "EPSG:3006",
        "-nln", table_name,
        "-nlt", "MULTILINESTRINGZ",
        "-dim", "3",
        "-overwrite",
        table_name        
    ]

    os.environ["PROJ_LIB"] = proj_lib_path
    os.environ["GDAL_DATA"] = gdal_data_path
    subprocess.run(ogr_command, check=True, capture_output=True, text=True)
    print(f"Importerade {wires_gdb} till PostGIS som tabellen '{table_name}'.")
        
wires_gdb = os.path.join(local_dir, f"wires_{run_ID}.gdb")
powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
powerlines_df.apply(convert_gdb_to_postGIS, axis=1)

Snyggt jobbat, nu har vi gjort det första steget med att konvertera linorna till PostGIS tabeller! 

Vill du se vad dessa steg resulterade i kan du ta dig en kik i verktyget pgAdmin som ska ha laddats ner på din dator när du installerade PostgreSQL. Fastna inte i detta steg, utan ta hjälp om du vill göra det.

[pgAdmin dokumentation](https://www.pgadmin.org/docs/) om du vill läsa mer


### Uppgift 3: Konvertera kantträd till PostGIS tabeller
1. Nästa kodblock slår samman alla kantträds-block tillhörande en lina. Detta görs precis som det gjorts tidigare. Kör detta block. 

In [ ]:
# Uppgift 3.1: Inga uppdateringar av koden krävs.
from pathlib import Path
import glob
import fileinput
import pandas as pd
import os

def combine_blocks(row):
    LG = row["LG"]
    line = row["line"]
    line_dir = Path(powerlines_folder) / LG / f"line_{line}"
    
    block_dir = os.path.join(line_dir, "kantträd", "block")
    combined_blocks_path = os.path.join(line_dir, "kantträd", "SKB_raw.txt")
    
    # Merge files for all blocks into one
    merge_blocks(block_dir, "*.txt", combined_blocks_path)
    print(f"Slog samman alla kantträd för {LG}/line_{line} till filen {combined_blocks_path}.")

def merge_blocks(src_dir, search_pattern, dst_file):
    blocks = glob.glob(os.path.join(src_dir, search_pattern))
    with open(dst_file, "w") as fh:
        input_lines = fileinput.input(blocks)
        fh.writelines(input_lines)

powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
powerlines_df.apply(combine_blocks, axis=1)
print(f"Alla linor har slagits samman")

Precis som tidigare så läses linorna in från en textfil innehållande alla linor (powerline_list). Istället för att skapa en feature class för vardera lina så ska vi nu skapa en PostGIS tabell. Detta gör du genom att implementera följande deluppgifter i metoden *create_SKB_table*.

2. Anslut till databasen *test_postgis_db*. Detta har du gjort i uppgift 1.3, så du kan kopiera den koden. 
3. Skapa en tom tabell och namnge den med namnet som finns i variabeln table_name. Följande attribut ska finnas med:

    | Name    | Data Type                        |
    |---------|----------------------------------|
    | objectid | SERIAL PRIMARY KEY              |
    | x       | DOUBLE PRECISION                 |
    | y       | DOUBLE PRECISION                 |
    | z       | FLOAT                            |
    | dz      | FLOAT                            |
    | mz      | FLOAT                            |
    | shape   | geometry(POINTZ, 3006)           |

    I detta fall är *geometry* en punkt i 3D där 3006 är Spatial Reference System Identifier som refererar till SWEREF 99 TM.  

4. Sätt in koordinaterna för kantträden i tabellen du just skapat. *shape* ska bestå av punkten (x, y, mz), psst glöm inte att ange 3006 som SRID. 
    

In [ ]:
import psycopg2
from pathlib import Path
import pandas as pd
import os

def create_SKB_table(row):
    LG = row["LG"]
    line = row["line"]
    table_name = f"{LG}_{line}_SKB_XYmZ"
    line_dir = Path(powerlines_folder) / LG / f"line_{line}"
    SKB_dir = line_dir / "kantträd"
    SKB_file_in = os.path.join(SKB_dir, f"SKB_raw.txt")

    # Uppgit 3.2

    # Uppgift 3.3

    with open(SKB_file_in, 'r') as src_file:
        for file_line in src_file:
            l_split = file_line.split(' ')
            x = float(l_split[0])
            y = float(l_split[1])
            z = float(l_split[2])
            dz = float(l_split[3])
            mz = z - dz

            # Uppgift 3.4

powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
powerlines_df.apply(create_SKB_table, axis=1)

Nu har du tagit dig igenom all implementation, snyggt jobbat! Förhoppningsvis har du hunnit bilda dig en någorlunda bild av hur det är att använda PostGIS och psycopg. Om inte, berätta det för mig så kan jag ha det i åtanke sedan när jag analyserar resultatet av funktionsanalysen. 

## Genomgång av förslag på arbetsflöde (PostGIS, psycopg)
*Tidsuppskattning: 30 minuter*

Nu ska vi gemensamt gå igenom hur hela flödet av kantträds-beräkningarna skulle kunna se ut. Syftet med detta är att ni ska kunna bilda er en bild av hur pass kompatibelt ni tror att psycopg och PostGIS är med resterande nuvarande flöde. Ställa gärna frågor under tiden så ska jag förtydliga. 

## Fyll i poängbladet (PostGIS, psycopg)
*Tidsuppskattning: 30 minuter*

Fyll nu tillsammans i poängbladet för PostGIS. Glöm inte att fylla i så mycket kommentarer som möjligt. Dessa ska reflektera de tankar som ni haft längs vägen och motivera poängen som sätts. 

## Filgeodatabas, arcpy
*Tidsåtgång: 20 minuter*

Nu går vi över till att utvärdera lösningen som bygger på filgeodatabas och arcpy. I och med att ni redan är familjära med arcpy och har implementerat vegetationsanalysen med hjälp av det så kommer detta segment att göra er påminda om hur detta var. Ni kommer att få några frågor att diskutera kring för att mjuka upp minnet. Parallellt med frågorna kan ni fylla i poängbladet med kommentarer.

### Finansiering 
1. Finns det några kostnader kopplat till användningen av arcpy? Om ja, hur ser dessa kostnader ut? 

*Nu kan funktionsset F1 fyllas i*

### Dokumentation
1. Läste du någon dokumentation under tiden vegetationsanalysen implementerades? Om ja, vilken dokumentation och hur var kvalitén av den?
2. Var det någon teknisk bit kopplat till arcpy som var svår att få grepp om? Hur fick du mer information om detta?

*Nu kan underfunktionen F2-SF04 fyllas i*

### Dataformat 
1. Vilka konverteringar mellan olika filformat måste göras för att kunna hantera data så som linor och träd? 
    - Genomförs dessa konverteringar smidigt eller är de implementerade på ett omständligt sätt där krångel brukar kunna uppstå?

*Nu kan funktionsset F3 fyllas i*

### Avståndsberäkningar
1. Hur är respektive beräkning implementerad: 
    - kortaste avståndet mellan trädets markpunkt och fas
    - kortaste avståndet mellan trädtopp och fas
    - horisontellt avstånd mellan träd och fas
2. Finns det några brister samt fördelar med dessa beräkningar?
3. För att sätta sig in i arcpy-metoderna som används vid distansberäkningarna, läste du någon dokumentation?
    - Vilken var denna dokumentation?
    - Hur var kvalitén av dokumentationen?
    - Täcktes alla aspekter som du funderade över upp i dokumentationen?

*Nu kan funktionsset F4 fyllas i*

### Datahantering
1. Hur ser implementationen ut för följande aktiviteter? Finns det några fördelar och/eller nackdelar med dessa?
    - Definiera SWEREF99 och RH2000 som spatial referens
    - Lägga till attribut till en feature class
    - Hantera fält-alias
    - Hantera domäner

*Nu kan funktionsset F5 fyllas i*

### Interoperabilitet
1. Brukar du visualisera och utforska data i kartläge i samband med vegetationsanalysen? 
    - Hur tycker du att det fungerar?
2. I vilken code editor har du jobbat med implementationen av scripten och varför?
    - Hur är din upplevelse av att koda i den code editorn?
    - Finns det någon funktion eller stöd som du saknar?
3. Hur är det att hantera virtuella miljöer och python-paket när du jobbar? 
4. Vilka python-paket används utöver arcpy i lösningen och hur har dessa paket lirat med varandra? 

*Nu kan resterande underfunktioner fyllas i*

## Avslutning
*Tidsuppskattning: 10 minuter*

Vilket jobb ni har gjort, tack snälla för er tid!

Det sista som nu återstår är att kolla igenom de två poängbladen och säkerställa att ni bedömt med samma glasögon i båda fallen av verktyg. 

### Feedback
Feedback på tillfället genomförs muntligt.